In [1]:
using JuMP
using DataFrames
using CSV
using Gurobi
using DataFrames
using Random
using LinearAlgebra
using Clustering

include("../LShaped.jl/src/structs.jl")
include("../LShaped.jl/src/setup.jl")
include("../LShaped.jl/src/algorithm.jl")

include("./parameters.jl")
include("./get_functions.jl")
include("./modification_functions_async.jl")

loadcsv = CSV.File("./LOAD.csv");
ptdfdf = DataFrame(CSV.File("./ptdfsmall.csv"));
loaddf = DataFrame(CSV.File("./loaddata.csv"))
winddf = DataFrame(CSV.File("./winddata.csv"))

part1458 = DataFrame(CSV.File("./scenarios/part1458.csv"))

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1317
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1317
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1317
┌ Info: Precompiling Clustering [aaaa29a8-35af-508c-8bc3-b662a17a0fe5]
└ @ Base loading.jl:1317


,1,2,3,4,5,6,7,8,9,10,11,12
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,4276,5742,1309,6698,2380,1506,3411,4314,6046,3962,5027,3331
2,1255,1776,6310,4467,5131,6530,1461,5377,4140,1485,5671,6497
3,4590,3466,1050,327,3884,2770,2657,3941,5520,5506,6242,6142
4,2658,1381,3786,960,5704,6388,2516,2907,3852,346,1594,2245
5,1056,6853,5795,5064,1204,3835,669,1533,3762,3144,608,5446


In [2]:
let 
    lmax = 0.0
    wmax = 0.0
    for i = 1:24
        #println(i, " ", maximum(loaddf[:,2+i]))
        if maximum(loaddf[:,2+i]) > lmax
            lmax = maximum(loaddf[:,2+i])
        end
        #println(maximum(winddf[:,2+i]))
        if maximum(winddf[:,2+i]) > wmax
            wmax = maximum(winddf[:,2+i])
        end
    end
    global lmax = lmax
    global wmax = wmax
end

loaddis = load_distribution_dict(loadcsv);

ptdfdict = Dict()

for i = 1:38
    br = ptdfdf[i,1]
    ptdfdict[br] = Dict()
    for j = 2:25
        bus = parse(Int64,names(ptdfdf)[j])
        ptdfdict[br][bus] = ptdfdf[i,j]
    end
end

lrts = 2850.0
wrts = 713.5

713.5

In [3]:
loadmat = lrts/lmax*1.35*transpose(Matrix(loaddf[:,3:26]))
windmat = wrts/wmax*transpose(Matrix(winddf[:,3:26]))

X = loadmat - windmat
X = 1/maximum(X)*X

24×6940 Matrix{Float64}:
 0.368347  0.393983  0.470704  0.285807  …  0.292437  0.418863  0.432446
 0.338962  0.393792  0.453791  0.28155      0.276893  0.403834  0.399128
 0.316082  0.402473  0.443553  0.281493     0.268886  0.404071  0.364005
 0.324778  0.411863  0.447889  0.285322     0.289251  0.400051  0.343051
 0.374886  0.426142  0.452954  0.296982     0.315441  0.393028  0.350853
 0.376164  0.455484  0.463445  0.324923  …  0.328632  0.277537  0.370265
 0.387638  0.49712   0.48287   0.363994     0.334217  0.301971  0.397744
 0.393537  0.516306  0.477814  0.382395     0.326832  0.325525  0.423494
 0.397756  0.515913  0.426702  0.37227      0.33891   0.345076  0.446025
 0.416539  0.518293  0.426805  0.354244     0.353091  0.368819  0.470047
 0.429368  0.51168   0.384735  0.335524  …  0.362596  0.389323  0.491079
 0.440492  0.495363  0.364886  0.314718     0.367803  0.405212  0.496611
 0.437593  0.475305  0.339491  0.294439     0.369218  0.412165  0.492898
 0.429052  0.466441  0.306

In [4]:
K = 12
exp = 1

trvec = collect(part1458[exp,:])

X1458 = Array{Float64}(undef, size(X,1), 1458)
for i = 1:1458
    X1458[:,i] = X[:, trvec[i]]
end
X1458

24×1458 Matrix{Float64}:
 0.368228  0.387624  0.425704  0.382545  …  0.581442  0.261847  0.224467
 0.34039   0.369311  0.417097  0.358996     0.575946  0.281765  0.26175
 0.33146   0.360269  0.409821  0.343517     0.570366  0.329337  0.303316
 0.318587  0.394246  0.406255  0.331917     0.571261  0.363465  0.314739
 0.325878  0.404872  0.41817   0.335646     0.568794  0.372744  0.322812
 0.350133  0.456595  0.435452  0.368649  …  0.577312  0.430074  0.35371
 0.39333   0.485518  0.449859  0.374354     0.602254  0.5014    0.413321
 0.448512  0.530736  0.479728  0.38641      0.618313  0.526661  0.43408
 0.49307   0.594089  0.507831  0.45188      0.634398  0.526189  0.422878
 0.51111   0.659406  0.564683  0.495549     0.64135   0.525492  0.417248
 0.526102  0.723057  0.623823  0.562197  …  0.650773  0.514509  0.414054
 0.560555  0.775579  0.675768  0.61267      0.650512  0.501093  0.411557
 0.581658  0.822575  0.713766  0.66497      0.662307  0.485701  0.402919
 0.617012  0.85374   0.73462 

In [5]:
centers = DataFrame(CSV.File("./scenarios/centers_$(K)_$(exp).csv"))

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.213372,0.424512,0.362769,0.237097,0.354508,0.488728,0.302932,0.317698
2,0.197601,0.4017,0.356696,0.225993,0.337067,0.485221,0.287083,0.314331
3,0.189753,0.384971,0.358258,0.222567,0.328197,0.485019,0.281851,0.314446
4,0.192454,0.380707,0.370273,0.227907,0.327285,0.488058,0.282968,0.318203
5,0.204661,0.391373,0.38893,0.248541,0.341185,0.504636,0.299577,0.329307
6,0.236503,0.413735,0.419959,0.286339,0.370338,0.543343,0.330713,0.358637
7,0.262982,0.43526,0.459345,0.317399,0.391004,0.590607,0.355834,0.396574
8,0.279662,0.472926,0.474312,0.337521,0.42131,0.602187,0.380952,0.409209
9,0.299342,0.516688,0.474588,0.362678,0.454588,0.584786,0.410289,0.407807


In [6]:
cldict = CSV.File("./scenarios/cldict_$(K)_$(exp).csv") |> Dict

Dict{Int64, String} with 12 entries:
  5  => "[4276, 6698, 2380, 5013, 4644, 4911, 6713, 2375, 3540, 1652, 3446, 931…
  12 => "[3411, 4716, 2250, 1457, 1061, 1913, 5077, 477, 679, 447, 78, 313, 519…
  8  => "[1506, 4314, 3331, 3639, 697, 368, 5900, 2823, 5480, 2853, 417, 4480, …
  1  => "[2663, 3333, 2152, 352, 314, 1010, 1901, 1741, 1537, 3017, 5172, 3929,…
  6  => "[4418, 6578, 6910, 6547, 6617, 2585, 3628, 5468, 4013, 6233, 5140, 692…
  11 => "[4407, 6248, 6827, 2069, 6521, 5638, 6069, 6503, 6618, 6111, 1701, 476…
  9  => "[5742, 6046, 6814, 6401, 4602, 3151, 4975, 4958, 6443, 6272, 4949, 676…
  3  => "[3297, 4451, 4724, 6512, 5837, 5792, 763, 5047, 4778, 6908, 6338, 5057…
  7  => "[3962, 5027, 1986, 4283, 6696, 3605, 1634, 5940, 4214, 5521, 6115, 526…
  4  => "[2871, 5397, 735, 6297, 5939, 3305, 3792, 264, 4109, 1946, 2940, 2668,…
  2  => "[1309, 3137, 5982, 3815, 1310, 6368, 2750, 4625, 3478, 6487, 6501, 388…
  10 => "[3680, 3381, 4339, 4529, 3168, 1771, 4374, 5890, 6879, 2205, 32

In [31]:
clid = 1
center = centers[:,clid]

24-element Vector{Float64}:
 0.21337223697346971
 0.19760120169996256
 0.18975342415186386
 0.19245399316139572
 0.20466107758692803
 0.2365034207977876
 0.26298154907795596
 0.2796618570136885
 0.2993415347118002
 0.3203711372211312
 0.33142570440317737
 0.34503828596109004
 0.3545399881426678
 0.36229104926581585
 0.3682982241678767
 0.3751055693532692
 0.37553255886805537
 0.36841013668934414
 0.3709518903591809
 0.36941357651466594
 0.3565635810288973
 0.33029087347670566
 0.2979290824559036
 0.2657004497636093

In [26]:
vect = Int64[];
n = length(cldict[clid])
string = cldict[clid][2:n-1]
for i = 1:length(string)
    if string[i] == ','
        if i-1 == 1
            push!(vect, parse(Int64, string[i-1]))
        elseif i-2 == 1
            push!(vect, parse(Int64, string[i-2:i-1]))
        elseif i-3 == 1
            push!(vect, parse(Int64, string[i-3:i-1]))
        elseif i-4 == 1
            push!(vect, parse(Int64, string[i-4:i-1]))
        elseif string[i-2] == ' ' 
            push!(vect,parse(Int64, string[i-1]))
        elseif string[i-3] == ' '
            push!(vect,parse(Int64, string[i-2:i-1]))
        elseif string[i-4] == ' '
            push!(vect,parse(Int64, string[i-3:i-1]))
        elseif string[i-5] == ' '
            push!(vect,parse(Int64, string[i-4:i-1]))
        else
            println("Uh oh!")
        end
    elseif i == n-2
        if i == 1
            push!(vect, parse(Int64, string[i]))
        elseif i-1 == 1
            push!(vect, parse(Int64, string[i-1:i]))
        elseif i-2 == 1
            push!(vect, parse(Int64, string[i-2:i]))
        elseif i-3 == 1
            push!(vect, parse(Int64, string[i-3:i]))
        elseif string[i-1] == ' ' 
            push!(vect,parse(Int64, string[i]))
        elseif string[i-2] == ' '
            push!(vect,parse(Int64, string[i-1:i]))
        elseif string[i-3] == ' '
            push!(vect,parse(Int64, string[i-2:i]))
        elseif string[i-4] == ' '
            push!(vect,parse(Int64, string[i-3:i]))
        else
            println("Uh oh!")
        end
            
    end
end


In [44]:
distvect = Vector{Float64}(undef, length(vect))
for i = 1:length(vect)
    dist = norm(center-X[:,vect[i]])
    println(i, " ",dist)
    distvect[i] = dist
end

1 0.29572059024620906
2 0.20480548971083443
3 0.23105418113781656
4 0.25324008001944165
5 0.35854275045963346
6 0.18264429772844434
7 0.20287960939348676
8 0.15942761487804563
9 0.21757989650239082
10 0.14544423849261193
11 0.1956931264050371
12 0.2165258759642695
13 0.13372529047459109
14 0.3029781184756237
15 0.2670021872817022
16 0.13678163039536126
17 0.18726618649281987
18 0.336172048326199
19 0.2838439123156306
20 0.23192359291611808
21 0.20949093917153247
22 0.1586369873248301
23 0.321955917643526
24 0.3063840587487942
25 0.08425698656949154
26 0.2242731409612391
27 0.22337699878701348
28 0.1941542098057764
29 0.23832564661754166
30 0.2928088218645808
31 0.2221493244141172
32 0.21160474530639672
33 0.17589293609846932
34 0.12599802423953127
35 0.30160162296528226
36 0.10168919947698112
37 0.26094364955095645
38 0.2923096550522519
39 0.14343936474002586
40 0.2803561111679081
41 0.1675611731602424
42 0.22833216484691177
43 0.19539846078834486
44 0.1419125189435753
45 0.16093236776

In [51]:
#vect[argmin(distvect)]
println(K, " ", exp, " ", clid, " ", vect[argmin(distvect)])

12 1 1 5593


In [49]:
X[:,vect[25]]

24-element Vector{Float64}:
 0.19456508827169108
 0.18668500501407745
 0.18596644308229723
 0.19413567230271273
 0.22049300350735457
 0.265129585478212
 0.27647002839610557
 0.2821409557971289
 0.29353815171442904
 0.307331792521837
 0.31884669604601623
 0.33155257461204884
 0.36318906322691263
 0.3775413447897579
 0.39366129750533896
 0.4126244292263645
 0.4005802654603189
 0.3923772298123138
 0.3774734497730241
 0.3872951317038367
 0.37026743299179976
 0.3279093033456326
 0.28030307850635044
 0.24619032934478335

In [50]:
minimum(distvect)

0.08425698656949154

In [55]:
for K in [12,24]
    for exp = 1:5

        trvec = collect(part1458[exp,:])

        X1458 = Array{Float64}(undef, size(X,1), 1458)
        for i = 1:1458
            X1458[:,i] = X[:, trvec[i]]
        end

        centers = DataFrame(CSV.File("./scenarios/centers_$(K)_$(exp).csv"))
        cldict = CSV.File("./scenarios/cldict_$(K)_$(exp).csv") |> Dict

        for clid = 1:K

            center = centers[:,clid]

            vect = Int64[];
            n = length(cldict[clid])
            string = cldict[clid][2:n-1]
            for i = 1:length(string)
                if string[i] == ','
                    if i-1 == 1
                        push!(vect, parse(Int64, string[i-1]))
                    elseif i-2 == 1
                        push!(vect, parse(Int64, string[i-2:i-1]))
                    elseif i-3 == 1
                        push!(vect, parse(Int64, string[i-3:i-1]))
                    elseif i-4 == 1
                        push!(vect, parse(Int64, string[i-4:i-1]))
                    elseif string[i-2] == ' ' 
                        push!(vect,parse(Int64, string[i-1]))
                    elseif string[i-3] == ' '
                        push!(vect,parse(Int64, string[i-2:i-1]))
                    elseif string[i-4] == ' '
                        push!(vect,parse(Int64, string[i-3:i-1]))
                    elseif string[i-5] == ' '
                        push!(vect,parse(Int64, string[i-4:i-1]))
                    else
                        println("Uh oh!")
                    end
                elseif i == n-2
                    if i == 1
                        push!(vect, parse(Int64, string[i]))
                    elseif i-1 == 1
                        push!(vect, parse(Int64, string[i-1:i]))
                    elseif i-2 == 1
                        push!(vect, parse(Int64, string[i-2:i]))
                    elseif i-3 == 1
                        push!(vect, parse(Int64, string[i-3:i]))
                    elseif string[i-1] == ' ' 
                        push!(vect,parse(Int64, string[i]))
                    elseif string[i-2] == ' '
                        push!(vect,parse(Int64, string[i-1:i]))
                    elseif string[i-3] == ' '
                        push!(vect,parse(Int64, string[i-2:i]))
                    elseif string[i-4] == ' '
                        push!(vect,parse(Int64, string[i-3:i]))
                    else
                        println("Uh oh!")
                    end

                end
            end

            distvect = Vector{Float64}(undef, length(vect))
            for i = 1:length(vect)
                dist = norm(center-X[:,vect[i]])
                #println(i, " ",dist)
                distvect[i] = dist
            end

            #vect[argmin(distvect)]
            println(K, ", ", exp, ", ", clid, ", ", vect[argmin(distvect)], ", ", minimum(distvect))
        end
    end
end

12, 1, 1, 5593, 0.08425698656949154
12, 1, 2, 3530, 0.05204891559938412
12, 1, 3, 5915, 0.07508964182438087
12, 1, 4, 4872, 0.07816014502784412
12, 1, 5, 1673, 0.0452266147101601
12, 1, 6, 1088, 0.10178597704254941
12, 1, 7, 4294, 0.0668841820094935
12, 1, 8, 47, 0.06052953765491866
12, 1, 9, 5700, 0.07203714585747725
12, 1, 10, 1052, 0.09521211800984859
12, 1, 11, 6097, 0.06595985665325658
12, 1, 12, 3719, 0.07649866627755296
12, 2, 1, 3684, 0.06582026367898099
12, 2, 2, 6409, 0.06606020808510557
12, 2, 3, 1480, 0.0702086062142019
12, 2, 4, 6122, 0.07402118716132047
12, 2, 5, 3938, 0.07485292726779592
12, 2, 6, 6914, 0.12818332240293726
12, 2, 7, 2324, 0.07298496488546867
12, 2, 8, 2817, 0.11585992601366779
12, 2, 9, 4945, 0.059549051159762084
12, 2, 10, 4896, 0.07851794812643526
12, 2, 11, 6634, 0.04604551558012723
12, 2, 12, 5497, 0.1110065936993079
12, 3, 1, 4882, 0.06269915648887998
12, 3, 2, 3374, 0.06793432191911347
12, 3, 3, 6142, 0.0546137821013896
12, 3, 4, 5577, 0.0631380571